# Lab 2 - Linear regression </br> M1 Data Science, ML1

# Introduction

In [1]:
# %%
# "IPython magic command" to sutomatically reload the imported packages after changes in a package
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
import sklearn

# plt.rc("font", **{"family": "sans-serif", "sans-serif": ["Helvetica"]})
# plt.rc("text", usetex=True)

SMALL_SIZE = 16
MEDIUM_SIZE = 20
BIGGER_SIZE = 24

plt.rc("font", size=SMALL_SIZE)  # default text size
plt.rc("axes", titlesize=SMALL_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=SMALL_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=SMALL_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title

# display backend for matplotlib
%matplotlib inline
# %matplotlib widget
# %matplotlib notebook

<div class="" style="margin-top: 0px">

Lab report written by: **Danila Pechenev**, **Gwenn Garrigues**.

</div>

# Exercise 1: linear regression

Let $N, M$ be two postive integers, and consider a dataset $\mathcal{D} = \{ (y_n \mathbf{x}_n) \}_{1 \leq n \leq N}$, with targets $\mathbf{y} = (y_n)_{1 \leq n \leq N} \in \mathbb{R}^N$, and (standardized) data $\mathbf{X} = [\mathbf{x}_1, \dotsc, \mathbf{x}_N]^T \in \mathbb{R}^{N \times M}$.

This exercise is aimed at performing linear regression with `numpy` and `sklearn` on the `California house` dataset, loaded below.

For the sake of simplicity, the dataset is NOT split into a train and a test set until question 5, where the quality of the model is properly assessed.

In [2]:
# loading the dataset
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()

# X = housing.data, y = housing.target
X = housing.data
y = housing.target

print(X.shape, y.shape)

print(housing.feature_names)
print(housing.DESCR)

(20640, 8) (20640,)
['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']
.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

:Number of Instances: 20640

:Number of Attributes: 8 numeric, predictive attributes and the target

:Attribute Information:
    - MedInc        median income in block group
    - HouseAge      median house age in block group
    - AveRooms      average number of rooms per household
    - AveBedrms     average number of bedrooms per household
    - Population    block group population
    - AveOccup      average number of household members
    - Latitude      block group latitude
    - Longitude     block group longitude

:Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in

In [3]:
features_min = np.min(X, axis=0)
features_max = np.max(X, axis=0)
range_features = np.abs(features_max - features_min)

with np.printoptions(precision=2):
    print(
        "Feature ranges: \n names: {} \n dr.  : {} \n min  : {} \n max  : {}".format(
            housing.feature_names, range_features, features_min, features_max
        )
    )

Feature ranges: 
 names: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude'] 
 dr.  : [1.45e+01 5.10e+01 1.41e+02 3.37e+01 3.57e+04 1.24e+03 9.41e+00 1.00e+01] 
 min  : [   0.5     1.      0.85    0.33    3.      0.69   32.54 -124.35] 
 max  : [ 1.50e+01  5.20e+01  1.42e+02  3.41e+01  3.57e+04  1.24e+03  4.20e+01
 -1.14e+02]


<div class="alert" style="margin-top: 0px">

1. Standardize the data using `sklearn`, and store the output into a variable `scaled_X`. Is it useful for this dataset? Justify your answer.

> Indication: take a look at [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) 

</div>

In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

In [5]:
scaler = StandardScaler()
scaler.fit(X)
scaled_X = scaler.transform(X)
scaled_X[0]

array([ 2.34476576,  0.98214266,  0.62855945, -0.15375759, -0.9744286 ,
       -0.04959654,  1.05254828, -1.32783522])

<div style="border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; background-color:#f0fff0">
We do that to avoid loss function dominated by some features (due to the different ranges of them). It is useful for this dataset because, as we've just seen, the columns of the dataset have different ranges. For example, the range of <b>Population</b> is 3.57e+04 and the range of <b>Latitude</b> is just 1.00e+01
</div>

<div class="alert" style="margin-top: 0px">

2. Write ERM problem corresponding to linear least-squares regression. Recall the form of the analytic expression solution, with the definition of the elements involved.

    > Indication: type your answer in $\LaTeX$ in the cell below, or include a picture of your handwritten answer.

</div>

<div style="border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; background-color:#f0fff0">

$$\beta^* \in \underset{\beta\in\mathbb{R}^{D+1}}{argmin}\frac{1}{2N}||y-\tilde{X}\beta||^2_2 \text{, with } \tilde{X} = \left[ \phi(x_1)^T, ... , \phi(x_N)^T \right]^T \in \mathbb{R}^{N \times(D+1)}$$
$$\text{where }\phi : R^D \to R^{D+1} : x \mapsto [1,x^T]^T.$$
$$\text{If rank}(\tilde{X}) = D + 1, \tilde{X}^T\tilde{X} \text{ is invertible, and}$$
$$\beta^* = (\tilde{X}^T\tilde{X})^{-1}\tilde{X}^Ty$$
</div>

<div class="alert" style="margin-top: 0px">

3. 
   1. Compute the solution to the problem with `numpy` from the analytic expression, calling the result `beta_numpy`.
   
   2. Compute the solution with `sklearn`, `beta_sklearn`. 

   3. Test whether the two array are significantly different. 

> Hints: 
> - the function [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve) and the class [`sklearn.linear_model.LinearRegression`](https://scikit-learn.org/stable/modules/linear_model.html) will be useful
> - the function [`numpy.allclose`](https://numpy.org/doc/stable/reference/generated/numpy.allclose.html#numpy-allclose) can be useful to test equality between two arrays.

</div>

In [6]:
X_wave = np.c_[X, np.ones(X.shape[0])]
beta_numpy = np.linalg.solve(np.transpose(X_wave) @ X_wave, np.transpose(X_wave) @ y)

In [7]:
model = LinearRegression()
model.fit(X, y)
beta_sklearn = np.append(model.coef_, model.intercept_)

In [8]:
np.allclose(beta_numpy, beta_sklearn)

True

<div style="border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; background-color:#f0fff0">
beta_numpy and beta_sklearn are the same
</div>

<div class="alert" style="margin-top: 0px">

4. Evaluate $\kappa(\widetilde{\mathbf{X}}^T\widetilde{\mathbf{X}})$, the condition number of the matrix $\widetilde{\mathbf{X}}^T\widetilde{\mathbf{X}}$ using the function [`numpy.linalg.cond`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.cond.html#numpy.linalg.cond). Is the least-squares problem considered well-conditioned? Justify your answer.

</div>

In [9]:
np.linalg.cond(np.transpose(X_wave) @ X_wave)

np.float64(56803951681.7832)

<div style="border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; background-color:#f0fff0">
The matrix is very ill-conditioned because cond_number >> 1 (even more than $10^{10}$), and inversion is unreliable 
</div>

<div class="alert" style="margin-top: 0px">

5. To properly assess the quality of the regressor, the dataset will now be split into a train and a test set. Complete the code below to:
    - split the dataset into a train and a test set
    - use `sklearn` to apply K-fold cross validation on the train set, and return the regressor associated with the best MSE criterion;
    - evaluate the performance on the test set in terms of the MSE criterion.

    > Indications: 
    > - if needed, take a look at the documentation of [`sklearn.model_selection.train_test_split`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html#train-test-split), and take some inspiration from [`sklearn` examples](https://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html#sphx-glr-auto-examples-linear-model-plot-ols-py).
    >
    > - the documentation and examples for [`Pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html) (or [`sklearn.pipeline.make_pipeline`](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.make_pipeline.html)) and [`sklearn.preprocessing.StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html) can be useful;
    >
    > - to return the regression coefficients from the pipeline, take a look its [named_steps](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.named_steps) method (if needed, see also the [stackoverflow answer](https://stackoverflow.com/questions/43856280/return-coefficients-from-pipeline-object-in-sklearn));
    >
    > - for the evaluation metrics, see [the documentation](https://scikit-learn.org/stable/modules/model_evaluation.html) and the module [sklearn.metrics](https://scikit-learn.org/stable/api/sklearn.metrics.html) 
</div> 

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import KFold, train_test_split
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a linear regression model
linear_regression = LinearRegression()

# Create scaler
scaler = StandardScaler()

pipe = Pipeline(steps=[("scaler", scaler), ("linearregression", linear_regression)])

# Set-up cross-validation
kf = KFold(n_splits=5)

fold_id = 0
for train_index, test_index in kf.split(X_train):
    X_subtrain, X_subtest = X_train[train_index], X_train[test_index]
    y_subtrain, y_subtest = y_train[train_index], y_train[test_index]

    # train the model using the scaled train set
    pipe.fit(X_subtrain, y_subtrain)

    # make predictions using the test set
    y_pred = pipe.predict(X_subtest)

    # display mean squared error
    print("Fold {}: MSE: {:.2f}".format(fold_id, mean_squared_error(y_subtest, y_pred)))
    # display mean absolute error
    print("        MAE: {:.2f}".format(mean_absolute_error(y_subtest, y_pred)))
    # display coefficient of determination: 1 is perfect prediction
    print("        R2 : {:.2f}".format(r2_score(y_subtest, y_pred)))
    fold_id += 1

Fold 0: MSE: 0.52
        MAE: 0.53
        R2 : 0.62
Fold 1: MSE: 0.50
        MAE: 0.53
        R2 : 0.61
Fold 2: MSE: 0.52
        MAE: 0.52
        R2 : 0.61
Fold 3: MSE: 0.51
        MAE: 0.52
        R2 : 0.61
Fold 4: MSE: 0.55
        MAE: 0.54
        R2 : 0.60


In [11]:
pipe.fit(X_train, y_train)
print(f"MSE on the test set: {mean_squared_error(y_test, pipe.predict(X_test))}")

MSE on the test set: 0.5558915986952442


# Exercise 2: least-squares and matrix inversion

Let $N, M$ be two positive integers, $\mathbf{A} \in \mathbb{R}^{N \times M}$ such that $\text{rank}(\mathbf{A}) = M$, and $\mathbf{y} \in \mathbb{R}^N$. 

This exercise is aimed at comparing the accuracy and speed of 2 approaches to compute $\widehat{\boldsymbol{\beta}} = (\mathbf{A}^T\mathbf{A})^{-1}\mathbf{A}^T\mathbf{y}$, solution to the problem 

\begin{equation*}
    \underset{\boldsymbol{\beta} \in \mathbb{R}^M}{\text{minimize}} \; \frac{1}{2} \| \mathbf{y} - \mathbf{A}\boldsymbol{\beta} \|_2^2
\end{equation*}

1. computing $\mathbf{B} = (\mathbf{A}^T\mathbf{A})^{-1}$, and then $\widehat{\boldsymbol{\beta}} = \mathbf{B}\mathbf{A}^T \mathbf{y}$.
2. solving the problem with an iterative solver, e.g., using [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve).

To do so, we will use the data generated below, knowing the ground truth value $\boldsymbol{\beta}^*$ to evaluate numerical accuracy.

In [12]:
rng = np.random.default_rng(1234)

N = 200
M = 100

A = rng.standard_normal(size=(N, M))
beta_true = rng.standard_normal(size=(M,))
y = A @ beta_true

<div class="alert" style="margin-top: 0px">

1. 
   - Compute $\widehat{\boldsymbol{\beta}}$ with the two approaches mentioned above, registering the time needed to solve the problem with the 2 version. To do so, complete the code cell below.
   - Compare the time required between the two approaches

> Indication: see the functions [`np.linalg.inv`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html#numpy.linalg.inv) and [`numpy.linalg.solve`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.solve.html#numpy.linalg.solve)

</div>

In [13]:
%%timeit
# Approach based on direct matrix inversion

B = np.linalg.inv(np.transpose(A) @ A)
beta_inv = B @ np.transpose(A) @ y

675 μs ± 362 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [14]:
%%timeit
# Approach based on iterative algorithm

beta_solve = np.linalg.solve(np.transpose(A) @ A, np.transpose(A) @ y)

141 μs ± 22.9 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [16]:
B = np.linalg.inv(np.transpose(A) @ A)
beta_inv = B @ np.transpose(A) @ y

beta_solve = np.linalg.solve(np.transpose(A) @ A, np.transpose(A) @ y)

<div class="alert" style="margin-top: 0px">

2. The accuracy of $\widehat{\boldsymbol{\beta}}$ can be assess with respect to the ground truth $\boldsymbol{\beta}^*$ in terms of the reconstruction signal to noise ratio (rSNR), expressed in dB, defined by

\begin{equation*}
    \text{rSNR}(\widehat{\boldsymbol{\beta}}, \boldsymbol{\beta}^*) = 10 \log_{10} \Big( \frac{\| \boldsymbol{\beta}^* \|_2^2}{\| \boldsymbol{\beta}^* - \widehat{\boldsymbol{\beta}} \|_2^2} \Big).
\end{equation*}

- Compute the rSNR for the solution `beta_inv` and `beta_solve` computed above, and indicate which of the two esimates is the most precise.
- Conclude on the best approach to adopt to solve linear systems of equations.

</div>

In [17]:
rsnb_beta_inv = 10 * np.log10(np.square(beta_true).sum() / np.square(beta_true - beta_inv).sum())
rsnb_beta_solve = 10 * np.log10(np.square(beta_true).sum() / np.square(beta_true - beta_solve).sum())

In [18]:
print('rSNR')
print(f"beta_inv: {rsnb_beta_inv}, beta_solve: {rsnb_beta_solve}")

rSNR
beta_inv: 296.72178630583676, beta_solve: 295.85707005005355


<div style="border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; background-color:#f0fff0">
We can see that the approaches are almost equally precise (the difference is negligible)
</div>

<div style="border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; background-color:#f0fff0">
The approach using numpy.linalg.solve is better than the one using np.linalg.inv because it works faster (and even a little bit more precise)
</div>

<div class="alert" style="margin-top: 0px">

3. Use `numpy` to compute the condition number $\kappa(\mathbf{A})$ of the matrix $\mathbf{A}$. What information does this value convey about the inversion problem considered?

> Indication: take a look at the documentation of the function [`numpy.linalg.cond`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.cond.html#numpy.linalg.cond).

</div>

In [19]:
np.linalg.cond(A)

np.float64(5.3385724263555785)

<div style="border: 2px solid #4CAF50; padding: 10px; border-radius: 8px; background-color:#f0fff0">
A is ~5 so it is considered to be well-conditioned. That means that any small perturbation, for instance, 1% error in data, could at most cause about 5% error in the solution because the relative error in the solution is bounded by the condition number times the relative error in the data
</div>

---

End of lab2.